In [8]:
# import package cells
import pandas as pd
import numpy as np
import re
import nltk
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from quantulum import parser

import xml.etree.ElementTree as ET
import spacy
from __future__ import unicode_literals
nlp = spacy.load('en_core_web_lg')
import re

77

In [ ]:


def is_keyword_found(line,keywords,threshold=80):
    """Finds whether any of the keywords match is found in the given line within the given threshold"""
    
    flag=False
    header='nill'
    for key in keywords:
        if  fuzz.ratio(key.lower(),line.lower().strip())>threshold:
            flag=True
            header=key
            
            break

    return [flag,header]

def find_entity(t):
    
    """Checks the text for country entity"""
    
    t=t.decode('utf8')
    if t.strip() in all_countries:
        #print('from country',t)
        return t.strip()
    if len(t)>=2:
        tokens =  nlp(t)       #print tokens.ents
        if len(tokens.ents)!=0:

            entity_text=''
            for entity in tokens.ents:
                #print 'label',entity.label_,'text',entity.text
                entity_label=entity.label

                if  entity.label_=='GPE' :
                    # print 'entities',entity.text
                    return entity.text

            if entity_text=='':

                return 'nill'
    else:
        return 'nill'
            
  

def remove_non_ascii_2(text):
    
    """Removes non ascii characters from the text (mainly to avoid encoding errors)""""
    if text==None:
        return
    else:
       return re.sub(r'[^\x00-\x7F]+',' ', text)  

def head_body_tagger(file_as_list,headers):
    """Returns dictionary which returns headers(keys) which are tagged with body(value)"""
 
    file_as_list=file_as_list+['EOF']  #to capture the last heading.
    
    header_body={}
    header='Title'
    header_body['Title']=[]
    body=[]
    
    for line_no in range(0,len(file_as_list)):
        flag,current_header=is_keyword_found(file_as_list[line_no],headers,threshold=85)
        if flag==False: #Appending to the body list if not the header
            
            body=body+[file_as_list[line_no]] 

        else: # Saving the body and resetting the header 
            print('header found:-',current_header,'---',file_as_list[line_no])

            header_body[header]=header_body[header]+[body]
             
            body=[]
            header=current_header
            header_body[header]=[]
            
    return header_body
    

def get_section(section_name,file_as_list):
    
    """Valid section names 
    Custody Headers,Fund  Accounting Headers,Fund Administration,Bundled Services,Others"""
    
    """Returns relevant section content based on the headers"""
    
    section_headers=pd.read_csv('/home/sriram/Desktop/AIDE/IE/Projects/ss/custody/eswar/keywords/header_tag_new.csv')
    all_headers=list(section_headers['Headers'])
    hb_tagged=head_body_tagger(file_as_list,all_headers)
    
    found_headers=hb_tagged.keys()
    
    relevant_headers=list(section_headers[section_headers['Section']==section_name]['Headers'])
    
    found_relevant_headers=[]
    for header in found_headers:
        if header in relevant_headers :
            found_relevant_headers.append(header)
    
    #print('Relevant Headers',relevant_headers)
    #print('Relevant Found Headers:-',found_relevant_headers)
    
    relevant_body=[]
    for header in found_relevant_headers:
        
        try:
            relevant_body=relevant_body+[header]+hb_tagged[header][0]
        except:
            relevant_body
        
    return relevant_body,found_relevant_headers,hb_tagged

 

def is_tiered(keyword_indices):
    """Check if upcoming lines from keywords lines contains tier values"""
    
    threshold=2
    match_lines=[]
    tier_keys=['million','billion','thereafter']
    for line_index in keyword_indices:
        sub_list=file_as_list[line_index:line_index+15]
        for sub_list_line in sub_list:

            if any(key in sub_list_line.lower() for key in tier_keys) :
                #print line_index,sub_list_line
                match_lines.append(sub_list_line)
            

    if len(set(match_lines))>=threshold: #for more than one incidence of co-occurence event.

        return True
    else:
        return False
    
def tier_extract(keyword_indices,file_as_list):
    """Extract tier lines from the text"""
    
    tier_keys=['million','billion','thereafter','Asset']
    basis_lis=[]
    for line_index in keyword_indices:
        sub_list=file_as_list[line_index:line_index+7]
        present_iter=0
        normal_iter=0
        last_iter_temp=[]
        for sub_list_line in sub_list:
            normal_iter=normal_iter+1
            #if any(key in sub_list_line.lower() for key in tier_keys) and len(re.findall(r'[^$]\s+\d{0,2}\.\d{1,4}',sub_list_line))>0:
            if any(key in sub_list_line.lower() for key in tier_keys) and len(re.findall(r'[^$]\s+\d\.\s*\d*',sub_list_line))>0:
     
                last_iter_temp=normal_iter
                print sub_list_line
                fraction=re.findall(r'\d*\/\d*',sub_list_line)
                if len(fraction)>0:
                    print('fraction_founded ........',sub_list_line)
                basis_lis.append(sub_list_line)
        if len(basis_lis)>0:
            a=0
            return remove_dup(basis_lis)
                
                
            
    return remove_dup(basis_lis)



def tier_extract_original(bps_list):
   """ Extract tier values [min,max,value] from the worded strings"""
    
   base_tier_keys=['first','$0']
   middle_tier_keys=['next','second','between']
   middle_tier_key_range=['to','between','and','-']
   top_tier_keys=['thereafter','excess','over']
   mult=1
   tier_structure=[]
    
   tier_limit=0;bps_value=0;tier_limit_temp=0

   for line in bps_list: 
        
      # print('line',line)
       try: 
           if '-' in line:
               val=re.findall(r'\$\s*(\d*\.*\,*\d*)',line)[1]
               tier_limit=float(val.replace(',',''))

           else:
               print line
               val=re.findall(r'\$\s*(\d*\.*\,*\d*)',line)[0]
               tier_limit=float(val.replace(',',''))
       except:
            tier_limit=tier_limit_temp
        
       #print('tier limit',tier_limit)
       #incase of 1/100 .3 basis point
       fraction=re.findall(r'\d\/\d*',line)
       if len(fraction)==2:
           print('fraction_founded ........')
           try:
               frac_mul=float(fraction[0].split('/')[0])/float(fraction[0].split('/')[1])
           except:
               print('fraction exception')
               frac_mul=1
       else:
           frac_mul=1
            
       #incase of percentage % in line
       if '%' in line:            
           bps_value=float(re.findall(r'[^$]\s+(\d{0,1}\.\s*\d*)',line)[0].replace(' ',''))*(1/100.0)*frac_mul #percentage and fraction mul.
       else:
           bps_value=float(re.findall(r'[^$]\s+(\d{0,1}\.\s*\d*)',line)[0].replace(' ',''))*frac_mul #only fraction mul.
        
    
       if 'billion' in line.lower():
           mult=1000
       else:
           mult=1
            
       if any(key in line.lower() for key in base_tier_keys):
           min_value=0
           max_value=tier_limit*mult
           print('max',max_value)
           tier_limit_temp=tier_limit_temp+max_value
           print('base')
            
       elif any(key in line.lower() for key in middle_tier_keys)  or ('over' in line.lower() and '-' in line.lower()):
           
           print('middle',line)
           min_value=tier_limit_temp
           max_value=tier_limit*mult+tier_limit_temp
           #tier_limit_temp+=tier_limit
           tier_limit_temp=max_value
            
           if any(key in line.lower() for key in middle_tier_key_range):
               min_value=float(re.findall(r'\$\s*(\d*)',line)[0])*mult
               max_value=float(re.findall(r'\$\s*(\d*)',line)[1])*mult
               tier_limit_temp+=max_value
        
                
       else:
           if 'over' not in line.lower() and 'excess' not in line.lower():
               min_value=tier_limit_temp*mult
           else:
                min_value=tier_limit*mult
                
           max_value='Above'
           print('top tier')
       try:
           print(bps_list)
           print(min_value,max_value,bps_value)
           tier_structure.append([min_value,max_value,bps_value])
       except:
           continue;
            
   return tier_structure

def remove_dup(bps_list):
    
    """Remove duplicate entries from the list"""
    fin_list = [] 
    for txt in bps_list: 
        if txt not in fin_list: 
            fin_list.append(txt) 
            
    return fin_list

def keyword_found_indices(file_as_list,keywords):
    """"Returns the indices of the file where the keywords are as substring of a line"""
    
    found_indices=[]

    for i in range(len(file_as_list)):
        
        if any(key in file_as_list[i] for key in keywords):
            
            found_indices.append(i)
    return found_indices

def keyword_found_indices2(file_as_list,keywords):
    """"Returns the indices of the file where the keywords are as substring of a line"""
    
    found_indices=[]

    for i in range(len(file_as_list)):
        
        if is_keyword_found(file_as_list[i].strip(),keywords,threshold=75)[0]==True:
            
            found_indices.append(i)
    return found_indices

def is_keyword_found(line,keywords,threshold=80):
    """Finds whether any of the keywords match is found in the given line within the given threshold"""
    
    flag=False
    header='nill'
    for key in keywords:
        if  fuzz.ratio(key.lower().strip(),line.lower().strip())>threshold:
            flag=True
            header=key
            return [flag,header]
    return False,'nill'
            
         

def find_nearby_quantity(found_indices,file_as_list,dtype,false_keys=[],window=5):
    """Finds the nearby numerical quantity of dtype(currency/percentage) in the given window of 
        the indices where keywords are found"""
    
    basis_lis='NA'
    cdollar_keywords=['C$','CAD','Can$']
    false_keys=[]
    fl=0
    currency_type=''
    for line_index in found_indices:
            
          
            sub_list=file_as_list[line_index:line_index+window]
            #print(sub_list[0])
            
            if 'no charge' in sub_list[0].lower():
                return 'No Charge'
            if 'waived' in sub_list[0].lower():
                return 'Waived'
            
            
            for i in range(len(sub_list)):

                #if any(key in sub_list[i].lower() for key in bps_keys):
                  
                for k in range(i,len(sub_list)):
                    bullets=re.findall(r"^\d{1}[.][^\d]",sub_list[k])
                    quants=[]
                    try:
                        quants=parser.parse(sub_list[k])
                       # print('quants',quants)
                    except:
                        #expression like these 44.0/44 throw quantum error..so capturing those lines 
                        print('Quants exception',sub_list[k]) 
                        #basis_lis.append(sub_list[k]) #first instance
                        return 'NA'

                    for q in range(len(quants)):  

                        if dtype=='percentage':
                            if len(bullets)==0 and len(parser.parse(sub_list[k]))>0 and ('dollar' not in parser.parse(sub_list[k])[q].unit.name)  and any(key not in sub_list[k].lower() for key in false_keys):
                                #print(parser.parse(sub_list[k])[0])
                                basis_lis.append(parser.parse(sub_list[k])[0]) #first instance

                                fl=1
                                break

                        elif dtype=='currency':
                            
                            
                            if len(bullets)==0 and len(parser.parse(sub_list[k]))>0:
                                print('currency found')
                                if any(key in file_as_list[i] for key in cdollar_keywords):
                                    currency_type='Canadian Dollar'
                                else:
                                    currency_type='US Dollar'
                                    
                                basis_lis=parser.parse(sub_list[k])[0].value #first instance
                                
                                return basis_lis



In [ ]:
#xml  parsing for tables from OCR output
country_list=[]
transaction_fee=[]
basis_point=[]
dois=[]
missed=[]
files=os.listdir('.//xmls2')
n=0
for doi in files:

    tree = ET.parse("/home/sriram/Desktop/AIDE/IE/Projects/ss/custody/eswar/xml/xmls2/"+doi)
    #tree=ET.parse("/home/sriram/Desktop/AIDE/IE/Projects/ss/custody/eswar/xml/ss_out_without_unlined/out/Bi2i_0000.xml")
    root = tree.getroot()
    no_pages=len(root)
    print 'n',n,doi
    n=n+1

    for page in range(no_pages):
        #print page
        for i in range(len(root[page][0])):

            if root[page][0][i].tag=='line':
               # print root[page][0][i][0]
                basis_pt=[]
                transaction_fe=[]



            if root[page][0][i][0].text!=None:
                text=remove_non_ascii_2(root[page][0][i][0].text)
                text_found=find_entity(text)
                if text!=None and text_found!='nill' and text_found!=None:
                    country_list.append(text_found)
                    dois.append(doi)
                    #print text_found
                    text_next=remove_non_ascii_2(root[page][0][i+1][0].text)

                    if text_next!=None and '.' in text_next:  #if basis point is found next
                        basis_pt=text_next

                        text_next_next=root[page][0][i+2][0].text
                        if text_next_next!=None and '$' in text_next_next:
                            transaction_fe=remove_non_ascii_2(text_next_next)

                    elif text_next!=None and '$' in text_next:  #if transaction charge is found next
                        transaction_fe=text_next

                        text_next_next=root[page][0][i+2][0].text
                        if text_next_next!=None and '.' in text_next_next:
                            basis_pt=remove_non_ascii_2(text_next_next)



                    if basis_pt!=[]:

                        basis_point.append(basis_pt)
                    else:
                        basis_point.append('Nill')
                    if transaction_fe!=[]:
                        transaction_fee.append(transaction_fe)
                    else:
                        transaction_fee.append('Nill')